In [ ]:
figsize('inline_short')

from collections import OrderedDict
import glob
import json
from typing import List

from skm import SKM

from features.util import *

# SKM example

In [ ]:
# Generate training data
#   - skm expects data of the shape (freqs, patches)
npoints = 200
X_train = dict(
    a=np.array(polar_to_cart(
        r=5 + np.random.normal(size=npoints, scale=1),
        theta=np.pi/4.0 + np.random.normal(size=npoints, scale=np.pi/8.0),
    )),
    b=np.array(polar_to_cart(
        r=7 + np.random.normal(size=npoints, scale=2),
        theta=-np.pi/4.0 + np.random.normal(size=npoints, scale=np.pi/32.0),
    )),
    c=np.array(polar_to_cart(
        r=5 + np.random.normal(size=npoints, scale=1),
        theta=5/4.0  * np.pi + np.random.normal(size=npoints, scale=np.pi/8.0),
    )),
)
display(
    X_train['a'].shape,
    X_train['b'].shape,
    X_train['c'].shape,
)
plt.plot(X_train['a'][0], X_train['a'][1], 'b.')
plt.plot(X_train['b'][0], X_train['b'][1], 'r.')
plt.plot(X_train['c'][0], X_train['c'][1], 'g.')
plt.show()

In [ ]:
# Combine into one unlabeled dataset and shuffle
X_train_nolab = np.concatenate((X_train['a'].T, X_train['b'].T, X_train['c'].T))
np.random.shuffle(X_train_nolab)
X_train_nolab = X_train_nolab.T
display(X_train_nolab.shape)
plt.plot(X_train_nolab[0,:], X_train_nolab[1,:], 'b.')
plt.show()

# Learn features
- Fit SKM centroids from the unlabeled version of the training data
- This produces an over-complete basis that we will use for the classification feature vectors

In [ ]:
skm = SKM(
    k=10,
    # variance_explained=0.99,
    # max_epocs=100,
    # assignment_change_eps=0.01,
    # standardize=False,
    # normalize=False,
    # do_pca=True,
    # pca_whiten=True,
    # visualize=False,
)

%time skm.fit(X_train_nolab)
display(
    'Fitted centroids:',
    skm.D.shape,
    skm.D,
    'Fitted whitening PCA:',
    skm.pca,
    skm.pca.components_,
)

skm_visualize_clusters(skm, X_train_nolab)
plt.show()


# Inspect the PCA whitening that SKM just did
- Whiten = decorrelate + standardize

In [ ]:
# Redo the whitening from skm.fit
X_train_nolab_white = skm.pca.transform(X_train_nolab.T).T

In [ ]:
# The whitened feature cov matrix should be ~I
for subplot, X in [
    (121, np.cov(X_train_nolab)),
    (122, np.cov(X_train_nolab_white)),
]:
    display(X)
    plt.subplot(subplot)
    plt.pcolormesh(X, cmap=mpl.cm.Greys_r)
    plt.axis('off')
plt.show()

In [ ]:
# The whitened data should have "balanced" feature dimensions
plt.subplot(121); plt.plot(X_train_nolab[0,:], X_train_nolab[1,:], 'k.')
plt.subplot(122); plt.plot(X_train_nolab_white[0,:], X_train_nolab_white[1,:], 'r.')
plt.show()

# Project test data onto the fitted skm centroids

In [ ]:
# Generate some test data
X_test = np.array(polar_to_cart(
    r=5 + np.random.normal(size=200, scale=1),
    theta=np.pi/4.0 + np.random.normal(size=npoints, scale=np.pi/8.0),
))
display(X_test.shape)
plt.plot(X_test[0], X_test[1], 'b.')
plt.show()

In [ ]:
# Project the test data using skm.transform, which:
#   - PCA whitens the test data, using the learned PCA from the (unlabeled) training data
#   - Projects the test data to the learned dictionary skm.D (i.e. the cluster centroids)
X_test_proj = skm.transform(X_test)
display(
    X_test.shape,  # n_freqs x n_patches
    X_test_proj.shape,  # n_centroids x n_patches
)
plt.subplot(121); plt.plot(X_test[0], X_test[1], 'b.')  # All 2 freq dims of the test data
plt.subplot(122); plt.plot(X_test_proj[0], X_test_proj[1], 'b.')  # Just the first two dims of X_test_proj (= first 2 PCs)
plt.show()

In [ ]:
# Summarize the patches (= time = samples) dimension for each cluster centroid (= over-completion of freqs)
#   - Reduces dimensionality
#   - Forgets time structure (we don't need to mess with alignment -- but we do lose sequential structure of songs)

def agg_over_time(X: 'np.ndarray[n_centroids, n_patches]', aggs: List[str]) -> 'pd.DataFrame[n_centroids, n_aggs]':
    '''
    Aggregate each centroid row X[i,:] using each agg function ('mean', 'std', 'min', 'max', etc.)
    '''
    return pd.DataFrame(OrderedDict({
        agg: {
            'mean':     lambda X: np.mean(X, axis=1),
            'std':      lambda X: np.std(X, axis=1),
            'min':      lambda X: np.min(X, axis=1),
            'max':      lambda X: np.max(X, axis=1),
            'median':   lambda X: np.median(X, axis=1),
            'skewness': lambda X: scipy.stats.skew(X, axis=1),
            'kurtosis': lambda X: scipy.stats.kurtosis(X, axis=1),
            'dmean':    lambda X: np.mean(np.diff(X, axis=1), axis=1),
            'dstd':     lambda X: np.std(np.diff(X, axis=1), axis=1),
            'dmean2':   lambda X: np.mean(np.diff(np.diff(X, axis=1), axis=1), axis=1),
            'dstd2':    lambda X: np.std(np.diff(np.diff(X, axis=1), axis=1), axis=1),
        }[agg](X)
        for agg in aggs
    }))

# A friendly df of the aggregated centroid features [make tidy?]
#   - (n_centroids, n_patches) -> (n_centroids, n_aggs)
X_test_proj_agg_df = agg_over_time(X_test_proj, [
    'mean',
    'std',
    'max',
])

# The raw feature vector, which should finally be amenable to vanilla classification
#   - (n_centroids, n_patches) -> (n_centroids * n_aggs,)
X_test_proj_agg_flat = X_test_proj_agg_df.T.values.flatten()

display(
    X_test_proj.shape,
    X_test_proj_agg_df.shape,
    X_test_proj_agg_df,
    X_test_proj_agg_flat.shape,
    X_test_proj_agg_flat,
)

# Classify

In [ ]:
# Load audio paths
audio_paths = pd.DataFrame([
    OrderedDict(
        source=path.split('/')[-4],
        species_code=path.split('/')[-3],
        title=os.path.splitext(path.split('/')[-1])[0],
        path=path,
    )
    for path in glob.glob(f'{peterson_dir}/*/audio/*')
])
display(
    audio_paths.shape,
    audio_paths[:5],
    audio_paths.groupby(['source', 'species_code']).count(),
)

In [ ]:
# Load audio from paths
recs_2ch = (audio_paths
    # [lambda df: df.species_code == 'wlswar'].reset_index(drop=True)  # For faster dev
    # [:5]  # For faster dev
    .assign(audio=lambda df: df.reset_index(drop=True).apply(axis=1, func=lambda rec:
        (
            # TODO Extract a reusable progress function (e.g. for next cell)
            print(f'Loading audio {rec.name + 1}/{len(df)}: {rec.path}') if rec.name % (np.ceil(len(df) / 5) or 1) == 0 else None,
            audiosegment.from_file(rec.path),
        )[-1]
    ))
)
display(
    recs_2ch.shape,
    recs_2ch.audio[:5],
)

In [ ]:
recs = (recs_2ch
    .assign(
        # Merge stereo to mono so we don't get confused when handling samples (we don't care about stereo vs. mono)
        audio=lambda df: df.audio.apply(lambda audio:
            audio.resample(channels=1, sample_rate_Hz=standard_sample_rate_hz)
            # audio.set_channels(1)  # TODO Any loss in fidelity by using .resample(channels=1)?
        ),
    )
    .assign(
        # Materialize audio samples
        samples=lambda df: df.audio.map(lambda audio: audio.to_numpy_array()),
    )
    .pipe(df_reorder_cols, last=['path'])
)
display(
    recs.shape,
    recs[:5],
)

In [ ]:
# Names for easier dev (better autocomplete)
rec0 = recs.iloc[0]
audio0 = rec0.audio

In [ ]:
def agg_over_time(X: 'np.ndarray[k, p]', aggs: List[str]) -> 'pd.DataFrame[k, a]':
    '''
    Aggregate each centroid row X[i,:] using each agg function ('mean', 'std', 'min', 'max', etc.)
    '''
    return pd.DataFrame(OrderedDict({
        agg: {
            'mean':     lambda X: np.mean(X, axis=1),
            'std':      lambda X: np.std(X, axis=1),
            'min':      lambda X: np.min(X, axis=1),
            'max':      lambda X: np.max(X, axis=1),
            'median':   lambda X: np.median(X, axis=1),
            'skewness': lambda X: scipy.stats.skew(X, axis=1),
            'kurtosis': lambda X: scipy.stats.kurtosis(X, axis=1),
            'dmean':    lambda X: np.mean(np.diff(X, axis=1), axis=1),
            'dstd':     lambda X: np.std(np.diff(X, axis=1), axis=1),
            'dmean2':   lambda X: np.mean(np.diff(np.diff(X, axis=1), axis=1), axis=1),
            'dstd2':    lambda X: np.std(np.diff(np.diff(X, axis=1), axis=1), axis=1),
        }[agg](X)
        for agg in aggs
    }))

def featurize(
    recs,
    f_min=1000,
    mel_bins=40,
    hop_length=256,
    frame_length=512,
    frame_window='hann',
    patch_length=4,
    pca_whiten_var=.99,
    k=512,
    aggs=['mean', 'std', 'max'],
    verbose=False,
) -> 'np.ndarray[k * n_aggs]':
    """
    Featurization pipeline

    Params
        |                  | defaults      | [SP14]         | [SBF16]       |
        |------------------|---------------|----------------|---------------|
        | sample_rate      | 22050         | 44100          | 22050
        | f_min            | 1000          | 500            | 2000
        |   f_max          | 11025         | 22050          | 11025
        | mel_bins (f)     | 40            | 40             | 40
        | hop_length       | 256 (12ms)    | 1024 (23ms)    | 32 (1.5ms)
        | frame_length     | 512 (23ms)    | 1024 (23ms)    | 256 (12ms)
        |   overlap_frac   | .5            | 0              | .875
        |   t/s            | 86            | 43             | 689
        | frame_window     | hann          | hamming        | hann
        | norm             | [TODO]        | RMS+median     | [TODO]
        | patch_length (p) | 4 (46ms)      | ~4 (~93ms)     | ~16 (~22ms)
        | pca_whiten_var   | .99           | —              | .99
        | k                | 500           | 500            | ~512
        | aggs             | μ,σ,max       | μ,σ            | ~μ,σ,max
        |   a              | 3             | 2              | ~3
        |   features       | 1500          | 1000           | ~1536

    Pipeline
        | audio      | (samples,)| (22050/s,)    | (44100/s)      | (22050/s,)
        | melspectro | (f, t)    | (40, 86/s)    | (40, 43/s)     | (40, 689/s)
        | patches    | (f, p, t) | (40, 4, 86/s) | (40, ~4, 43/s) | (40, ~16, 689/s)
        | projected  | (k, t)    | (500, 86/s)   | (500, 43/s)    | (~512, 689/s)
        | aggregated | (k, a)    | (500, 3)      | (500, 2)       | (~512, ~3)
        | features   | (k*a,)    | (1500,)       | (1000,)        | (~1536,)
    """

    (rec, audio, x, sample_rate) = unpack_rec(rec_or_audio_or_signal)

    # Show all input params and derived params back to the user
    _g = lambda x: '%.3g' % x
    _samples = sample_rate
    _f = mel_bins
    _t_s = '%s/s' % _g(sample_rate / hop_length)
    _t = int(sample_rate / hop_length * audio.duration_seconds)
    _p = patch_length
    _k = k
    _a = len(aggs)
    _ka = k * len(aggs)
    if verbose:
        print(f'audio:\n  {audio.name}')
        print('\nParams:')
        print(pd.Series(OrderedDict({
            'audio': repr('...%s' % audio.name[-20:]),
            '  channels': '%s ch' % audio.channels,
            '  sample_width': '%s bit' % (audio.sample_width * 8),
            '  sample_rate': '%s Hz' % sample_rate,
            '  duration': '%s s' % _g(audio.duration_seconds),
            'f_min': '%s Hz' % f_min,
            '  f_max': '%s Hz' % (sample_rate // 2),
            'mel_bins (f)': mel_bins,
            'hop_length': '%s (%s ms)' % (hop_length, _g(1 / sample_rate * hop_length * 1000)),
            'frame_length': '%s (%s ms)' % (frame_length, _g(1 / sample_rate * frame_length * 1000)),
            '  overlap_frac': _g(1 - hop_length / frame_length),
            '  t/s': _t_s,
            '  t': _t,
            'frame_window': repr(frame_window),
            'norm': '[TODO]',  # TODO
            'patch_length (p)': '%s (%s ms)' % (_p, _g(1 / sample_rate * _p * 1000)),
            'pca_whiten_var': pca_whiten_var,
            'k': k,
            'aggs': repr(aggs),
            '  a': _a,
            'features': k * _a,
        })))
        print('\nPipeline:')
        print(pd.Series(OrderedDict({
            'audio':      f'({_samples},)  (samples,)',
            'melspectro': f'({_f}, {_t_s})      (f, t)',
            'patches':    f'({_f}, {_p}, {_t_s})   (f, p, t)',
            'projected':  f'({_k}, {_t_s})      (k, t)',
            'aggregated': f'({_k}, {_a})      (k, a)',
            'features':   f'({_ka},)      (k*a,)',
        })))
        print()

    # audio (samples,) -> melspectro (f,t)
    # TODO
    ... = melspectro(rec, nperseg=..., overlap=..., ...)

    # melspectro (f,t) -> patches (f,p,t)
    # TODO

    # patches (f,p,t) -> projected (k,t)
    #   - TODO The above was written for 1 rec, but we need to take N recs for batch training, for skm.fit(...)
    #   - TODO Think about different handling of train vs. test (e.g. skm)
    if train:
        skm = SKM(k=10)
        skm.fit(recs)  # TODO Refactor to allow for this batch operation
        ...  # TODO
    else:
        projected = skm.transform(patches)

    # projected (k,t) -> aggregated (k,a)
    aggregated_df = agg_over_time(projected, aggs)

    # aggregated (k,a) -> features (k*a,)
    features = aggregated_df.T.values.flatten()

    return features

featurize(recs.iloc[0], verbose=True)

In [ ]:
recs.shape

In [ ]:
# TODO Need to return something from featurize
(recs
    [:5]
    .apply(axis=1, func=featurize, verbose=False)
)

In [ ]:
# # Example classifier
#
# from sklearn.neighbors import KNeighborsClassifier
#
# # TODO Audio files -> this stuff
# X_train = featurize(...)
# y_train = ...
# X_test = feature(...)
#
# knn = KNeighborsClassifier(3)
# knn.fit(X_train, y_train)
#
# knn.predict(X_test)
# knn.predict_proba(X_test)